## BoomBikes Case Study

<b>Problem Statement</b>: A US bike-sharing provider BoomBikes has recently suffered considerable dips in their revenues due to the ongoing Corona pandemic.
The purpose of this case study is to understand the factors on which the demand for these shared bikes depends. Specifically, they want to understand the factors affecting the demand for these shared bikes in the American market.

The company wants to know:
- Which variables are significant in predicting the demand for shared bikes.
- How well those variables describe the bike demands

In [1]:
# Importing all required packages
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm #to build linear model
import sklearn

from   sklearn.model_selection import train_test_split
from   sklearn.metrics import mean_squared_error
from   sklearn.metrics import r2_score
from   sklearn.preprocessing import StandardScaler, MinMaxScaler
from   sklearn.preprocessing import MinMaxScaler
from   sklearn.feature_selection import RFE
from   sklearn.linear_model import LinearRegression
from   statsmodels.stats.outliers_influence import variance_inflation_factor
pd.set_option('display.max_columns', None)
%matplotlib inline

### Step 1 - Reading and inspecting the data set

In [2]:
bike = pd.read_csv('day.csv')
bike.head()

FileNotFoundError: [Errno 2] File day.csv does not exist: 'day.csv'

In [ ]:
bike.shape

The Data set has 730 rows and 16 columns

In [ ]:
bike.info()

Apart from one column (which is an object), all the other columns are int/float

In [ ]:
bike.describe()

### Step 2 - Data Cleaning

-  ####  Subtask 2.1: Checking for missing values

In [ ]:
# Check the percentage of missing values in each column
round(100*(bike.isnull().sum()/len(bike.index)), 2).sort_values(ascending=False)

In [ ]:
# Check the percentage of missing values in each row
round((bike.isnull().sum(axis=1)/122)*100,2).sort_values(ascending=False)

<b>No missing values in the dataset</b>

-  ####  Subtask 2.2: Drop unnecessary columns

- instant - Index is there by default, hence this column is not required.
- dteday - year, month and weekday are available as separate columns hence we can drop this.
- casual - This data is included in the "cnt" columns, hence can be dropped
- registered - This data is included in the "cnt" columns, hence can be dropped

In [ ]:
bike_final = bike[['season', 'yr', 'mnth', 'holiday', 'weekday','workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed','cnt']]
bike_final.head()

-  ####  Subtask 2.3: Converting some columns to Categorical type

We notice that there are some numeric columns (season, mnth,weekday and weathersit), but in actual these should be categorical. Hence converting them.

In [ ]:
bike_final['season'] = bike_final['season'].map({1:'spring',2:'summer', 3:'fall', 4:'winter'})
bike_final['mnth'] = bike_final['mnth'].map({1:'Jan',2:'Feb', 3:'Mar', 4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sept',10:'Oct',11:'Nov',12:'Dec'})
bike_final['weekday'] = bike_final['weekday'].map({0:'Sunday',1:'Monday',2:'Tuesday',3:'Wednesday',4:'Thursday',5:'Friday',6:'Saturday'})
bike_final['weathersit'] = bike_final['weathersit'].map({1:'Clear-Partlycloudy',2:'Mist-Cloudy',3:'LightSnow-lightRain-Thunderstorm',4:'HeavyRain-IcePallets-Thunderstorm'})

In [ ]:
bike_final.head()

### Step 3 - Data Visualization

#### Subtask 3.1 Visualising Categorical Variables

As you might have noticed, there are a few categorical variables as well. Let's make a boxplot for some of these variables.

In [ ]:
def boxplt_cvar(cvar,x):
    plt.figure(figsize=(20, 12))
    for i in range(0,len(cvar)):
        plt.subplot(2,3,i+1)
        sns.boxplot(x = cvar[i], y = x, data = bike_final)
    plt.xticks(rotation=90)
    plt.show()

cvar =['season','yr','holiday','weekday','workingday','weathersit']
boxplt_cvar(cvar,'cnt')

In [ ]:
sns.boxplot(x = 'mnth', y = 'cnt', data = bike_final)

#### Observations:

- <b>Season</b>: Most of the bike bookings happened in the "fall" season followed by summer and winter
- <b>Year</b>: Almost 99% of the bike booking were increased from the year 2018 to 2019.
- <b>Holiday</b>: Most of the bike booking were happening when it was not a holiday, hence holiday cannot be a good predictor for the dependent variable
- <b>Weekday</b>: Number of bookings on a weekday are almost similar.This variable can have some or no influence towards the predictor.
- <b>WorkingDay</b>: Working days(neither holiday nor weekend) had more number of bookings as compared non-working days.
- <b>Weathersit</b>: Most of the bike booking were happening during 'Clear-Partlycloudy' followed by 'Mist-Cloudy'.
- <b>Mnth</b>: Month Jun to Sep is the period when bike demand is high. The Month Jan is the lowest demand month

#### Subtask 3.2 Visualising Numeric Variables

In [ ]:
# 
sns.pairplot(bike_final[[ 'temp','atemp', 'hum', 'windspeed','cnt']],diag_kind='kde')
plt.show()

<b>Observations:</b> The above Pair-Plot tells us that there is a LINEAR RELATION between 'temp','atemp' and 'cnt'

### Step 4: Data Preparation

-  ####  Subtask 4.1: Dummy variable

In [ ]:
# Create re-usable function to create dummy variables
def get_dummy(x,df):
    temp = pd.get_dummies(df[x], drop_first = True)
    df = pd.concat([df, temp], axis = 1)
    df.drop([x], axis = 1, inplace = True)
    return df
# Applying the function to the bikeSharing

bike_final = get_dummy('season',bike_final)
bike_final = get_dummy('mnth',bike_final)
bike_final = get_dummy('weekday',bike_final)
bike_final = get_dummy('weathersit',bike_final)
bike_final.head()

In [ ]:
bike_final.shape

-  ####  Subtask 4.2: Splitting the Data into Training and Testing Sets

In [ ]:
from sklearn.model_selection import train_test_split

# We use seed so that the train and test data set always have the same rows, respectively
np.random.seed(0)

df_train, df_test = train_test_split(bike_final, train_size = 0.7, test_size = 0.3, random_state = 100)

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
# Checking the correlation coefficients to see which variables are highly correlated

plt.figure(figsize = (30, 25))
sns.heatmap(df_train.corr(), annot = True, cmap="YlGnBu")
plt.show()

The <b>heatmap</b> clearly shows which all variable are multicollinear in nature, and which variable have high collinearity with the target variable.

-  ####  Subtask 4.3: Rescaling the features

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Applying scaler() to all the numeric variables

num_vars = ['temp', 'atemp', 'hum', 'windspeed','cnt']

df_train[num_vars]= scaler.fit_transform(df_train[num_vars])

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

-  ####  Subtask 4.4: Dividing into X and Y sets for the model building

In [ ]:
y_train = df_train.pop('cnt')
X_train = df_train

### Step 5: Building a linear model

-  ####  Subtask 5.1: RFE (Recursive Feature Elimination)

<b>Recursive feature elimination:</b> We will be using the LinearRegression function from SciKit Learn for its compatibility with RFE


In [ ]:
# Running RFE with the output number of the variable equal to 15
lm = LinearRegression()
lm.fit(X_train,y_train)
rfe = RFE(lm, 15)
rfe = rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
#Columns preferred to be used by the RFE process
X_train.columns[rfe.support_]

In [ ]:
#Columns NOT preferred to be used by the RFE process
X_train.columns[~rfe.support_]

In [ ]:
# Creating X_test dataframe with RFE selected variables
X_train_rfe = X_train[X_train.columns[rfe.support_]]
X_train_rfe.head()

-  ####  Subtask 5.2: Building a model using STATSMODEL

In [ ]:
# Creating re-usable functions to build model and calculate VIF
def lmodel(X,y):
    X = sm.add_constant(X) #Adding the constant
    lm = sm.OLS(y,X).fit() # fitting the model
    print(lm.summary()) # model summary
    return X
    
def calVIF(X):
    vif = pd.DataFrame()
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return(vif)

#### MODEL 1

In [ ]:
X_train_m1 = lmodel(X_train_rfe,y_train)

In [ ]:
#Calculating the Variance Inflation Factor
calVIF(X_train_m1)

##### p-value of Dec seems to be higher, hence dropping it.

In [ ]:
X_train_m1=X_train_m1.drop(["Dec"], axis = 1)

#### MODEL 2

In [ ]:
X_train_m1 = lmodel(X_train_m1,y_train)

In [ ]:
#Calculating the Variance Inflation Factor
calVIF(X_train_m1)

##### p-value of Jan seems to be higher than the significance value of 0.05, hence dropping it.

In [ ]:
X_train_m1=X_train_m1.drop(["Jan"], axis = 1)

#### MODEL 3

In [ ]:
X_train_m1 = lmodel(X_train_m1,y_train)

In [ ]:
#Calculating the Variance Inflation Factor
calVIF(X_train_m1)

##### p-value of Nov seems to be higher than the significance value of 0.05, hence dropping it.

In [ ]:
X_train_m1=X_train_m1.drop(["Nov"], axis = 1)

#### MODEL 4

In [ ]:
X_train_m1 = lmodel(X_train_m1,y_train)

In [ ]:
#Calculating the Variance Inflation Factor
calVIF(X_train_m1)

##### spring has high VIF and p value, so we will consider to drop

In [ ]:
X_train_m1=X_train_m1.drop(["spring"], axis = 1)

#### MODEL 4

In [ ]:
X_train_m1 = lmodel(X_train_m1,y_train)

In [ ]:
#Calculating the Variance Inflation Factor
calVIF(X_train_m1)

##### July has high p value, so we will consider to drop

In [ ]:
X_train_m1=X_train_m1.drop(["Jul"], axis = 1)

#### MODEL 5

In [ ]:
X_train_m1 = lmodel(X_train_m1,y_train)

In [ ]:
#Calculating the Variance Inflation Factor
calVIF(X_train_m1)

#### Observations:
- Now the model looks good with 10 variables and R-squared- 83.8 , Adj. R-squared- 83.4
- All VIFs are less than 2.
- All p-values are zero.
- The value of Prob (F-statistic) is almost zero.

### Step 6: Residual Analysis of the train data

In [ ]:
lm = sm.OLS(y_train,X_train_m1).fit()
y_train_cnt= lm.predict(X_train_m1)

In [ ]:
# Plot the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_cnt), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  # Plot heading 
plt.xlabel('Errors', fontsize = 18)                         # X-label

<b>All the error terms are normally distributed</b>

### Step 7:  Making Predictions Using the Final Model

#### Applying the scaling on the test sets

In [ ]:
#Scaling the test set
num_vars = ['temp','atemp', 'hum', 'windspeed', 'cnt']
df_test[num_vars] = scaler.fit_transform(df_test[num_vars])

In [ ]:
df_test.describe()

#### Dividing into X_test and y_test

In [ ]:
#Dividing into X and y
y_test = df_test.pop('cnt')
X_test = df_test

In [ ]:
# Using our model to make predictions.
X_train_m1 = X_train_m1.drop('const',axis=1)

# Creating X_test_new dataframe by dropping variables from X_test
X_test_m1 = X_test[X_train_m1.columns]

# Adding a constant variable 
X_test_m1 = sm.add_constant(X_test_m1)

#### Model Prediction

In [ ]:
# Making predictions
y_pred = lm.predict(X_test_m1)

#### Calculating R2

In [ ]:
r2=r2_score(y_test, y_pred)
print(r2)

#### Calculating Adjusted - R2

In [ ]:
n = X_test_m1.shape[0]   # n : Number of rows
p = X_test_m1.shape[1]   # p : No of predictors/ variables

In [ ]:
adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
adjusted_r2

In [ ]:
fig = plt.figure()
plt.scatter(y_test,y_pred)
sns.regplot(x=y_test, y=y_pred,line_kws={"color": "red"})
fig.suptitle('y_test vs y_pred', fontsize=20)              # Plot heading 
plt.xlabel('y_test', fontsize=18)                          # X-label
plt.ylabel('y_pred', fontsize=16) 

### Final Result Comparision

- R^2(Train Dataset) : 0.838

- Adjusted R^2(Train dataset) :  0.834

- R^2(Test Dataset) : 0.80396

- Adjusted R^2(Test dataset) :0.7935

This seems to be a good model

In [ ]:
print(lm.summary())


<b>We can see that the equation of our best fitted line is:</b>

$cnt= 0.2256 + (0.2289 * yr) - (0.0980 * holiday) + (0.5706 * temp) - (0.1740 * hum) - (0.1867 * windspeed) + (0.0895 * summer) + (0.1402 * winter) 
     + (0.1067 * Sept)  -(0.2367 * LightSnow-lightRain-Thunderstorm) - (0.0518 * Mist-Cloudy)$

### Hypothesis Testing

<b> H0:B1=B2=...=Bn=0</b>

<b> H1: at least one Bi!=0 for i = 1,2,3,....n</b>

Our final model has the below coefficients:
- yr                                   0.2289      
- holiday                             -0.0980      
- temp                                 0.5706      
- hum                                 -0.1740      
- windspeed                           -0.1867      
- summer                               0.0895      
- winter                               0.1402      
- Sept                                 0.1067     
- LightSnow-lightRain-Thunderstorm    -0.2367      
- Mist-Cloudy                         -0.0518  

It is evident that all our coefficients are non zero hence we <b> reject the NULL Hypothesis. </b>
    

### F-statistics


F-statistics is used for testing the overall significance of the model. Higher the F-Statistics value , more significant the model.

F-statistic : 257.6

Prob(F-statistic) : 7.80e-190

The F-Statistics value of 257.6 (which is greater than 1) and the p-value of '~0.0000' states that the overall model is significant.

### Conclusion

As per our final model, the top three predictor variables that influences the bike bookings are:
    
- <b>temp</b>: The users prefer to ride or rent a bike in a moderate temperature
- <b>year</b>: The company should encounter an increase in the number of users when the situation comes back to normal as compared to 2019.
- <b>season</b>: The company should focus on expanding its business in the Summer and the Fall season.
- <b>weathersit</b>:  The users prefer to rent a bike when the weather is clear and less cloudy. 
